In [ ]:
from hepmc.core.proposals import Gaussian
from hepmc import surrogate
from hepmc.core.density import Density
from hepmc.core import densities
from hepmc.core.hamiltonian.hmc import HamiltonianUpdate
from hepmc.core.hamiltonian.spherical_hmc import StaticSphericalHMC
from hepmc.core.hamiltonian.wall_hmc import WallHMC
from hepmc.core.sampling import Sample
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib notebook

In [ ]:
class Exponential(Density):
    def __init__(self):
        super().__init__(2, False)
        
    def pdf(self, xs):
        xs = np.atleast_2d(xs)
        return np.exp(-6*xs[:, 0])
        
    def pot(self, xs):
        xs = np.atleast_2d(xs)
        return 6*xs[:, 0]
    
    def pot_gradient(self, xs):
        xs = np.atleast_2d(xs)
        return np.broadcast_to([6, 0], xs.shape)

In [ ]:
target = Exponential()

In [ ]:
xvals = np.random.random((10000, 2))
xvals_df = pd.DataFrame(xvals)
fvals = target.pdf(xvals)

In [ ]:
g = sns.PairGrid(xvals_df)
g = g.map_diag(plt.hist, weights=fvals, bins=15)
g = g.map_offdiag(plt.hist2d, weights=fvals, bins=15)

In [ ]:
start = np.random.random(2)
print('start:', start)
print('pdf:', target.pdf(start))
print('pot:', target.pot(start))
print('pot_grad:', target.pot_gradient(start))
sampler = StaticSphericalHMC(target, .01, .01, 5, 5)
start = sampler.sample(10000, start).data[-1] # warmup
%time hmc_sample = sampler.sample(50000, start)

In [ ]:
hmc_sample_df = pd.DataFrame(hmc_sample.data)

In [ ]:
np.partition(hmc_sample.weights, -50)[-50:]

In [ ]:
k = sns.PairGrid(hmc_sample_df)
k = k.map_diag(plt.hist, bins=15, weights=hmc_sample.weights, range=(0, 1))
k = k.map_offdiag(plt.hist2d, bins=15, weights=hmc_sample.weights, range=[[0, 1], [0, 1]])

In [ ]:
start = np.random.random(2)
print('start:', start)
print('pdf:', target.pdf(start))
print('pot:', target.pot(start))
print('pot_grad:', target.pot_gradient(start))
sampler = WallHMC(target, .2, .2, 5, 5)
start = sampler.sample(5000, start).data[-1] # warmup
%time wallhmc_sample = sampler.sample(10000, start)

In [ ]:
wallhmc_sample_df = pd.DataFrame(wallhmc_sample.data)

In [ ]:
l = sns.PairGrid(wallhmc_sample_df)
l = l.map_diag(plt.hist, bins=15, range=(0, 1))
l = l.map_offdiag(plt.hist2d, bins=15, range=[[0, 1], [0, 1]])